In [1]:
import cv2
import numpy as np

In [2]:
net=cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

In [3]:
classes=[]
with open('labels.txt','r') as f:
    classes=f.read().splitlines()

In [4]:
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [5]:
img=cv2.imread('try3.jpeg')


In [6]:
height, width, _ = img.shape

In [7]:
blob=cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB=True, crop=False)

In [8]:
#cv2.imshow('Image', img)

In [9]:
net.setInput(blob)

## 

In [10]:
output_layer_names=net.getUnconnectedOutLayersNames()

In [11]:
layerOutputs=net.forward(output_layer_names)

In [12]:
boxes=[]
confidences=[]
class_ids=[]

In [13]:
for output in layerOutputs:
    for detection in output:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence > 0.5:
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            
            boxes.append([x, y, w, h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

In [14]:
print(len(boxes))

16


In [15]:
indexes=cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
print(indexes.flatten())

[ 1  9 14  7 10 13 15]


In [16]:
font=cv2.FONT_HERSHEY_PLAIN
colors=np.random.uniform(0, 255, size=(len(boxes),3))

In [17]:
for i in indexes.flatten():
    x, y, w, h= boxes[i]
    label=str(classes[class_ids[i]])
    confidence=str(round(confidences[i],2))
    color=colors[i]
    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
    cv2.putText(img, label+ " "+ confidence, (x,y+20), font, 2, (255,255,255), 2)

In [ ]:
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()